# H_22082024
# Analisis del volumen profile para encontrar zonas de baja cotización (FVG like) y como se comporta el precio en esas zonas.

En este Python Notebook analizaremos, con datos de mt5, como se comporta el mercado en las zonas previamente definidas.

## Conceptos clave del perfil de mercado:

1. **Oportunidad de precio en el tiempo (TPO):**
    
    representa los niveles de precio negociados durante intervalos de tiempo específicos. En un gráfico de perfil de mercado, las TPO indican la frecuencia con la que se negoció cada nivel de precio durante una sesión.
    
2. **Área de valor:**
    
    rango de precios en el que se produjo el 70 % de la actividad comercial durante una sesión. Esta área representa el valor justo percibido del mercado.
    
3. **Punto de control (POC):**
    
    el nivel de precio con el mayor volumen durante la sesión. El POC suele ser un nivel clave de soporte o resistencia.
    
4. **Balance inicial (IB):**
    
    rango de precios negociados durante la primera hora de la sesión de negociación. Proporciona una visión anticipada de la estructura del mercado del día.

In [52]:
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import pytz
from datetime import datetime
import matplotlib.pyplot as plt
import json
import plotly.express as px
import plotly.graph_objects as go

In [53]:
# Variables

TICKER = "6A_U"
STD_MULTIPLIER = 1

## 1. Definición de zonas de baja cotización para posterior analisis.
El primer paso a dar es definir como encontraremos las **zonas de baja cotización** en una sesion de mercado de forma objetiva.

A continuación, definiremos que es una zona de baja cotización para nuestro estudio:

1. Analisis basado en el *volumen profile*, el cual analiza el numero de cotizaciónes por precio que tiene lugar en una sesión de trading predefinida (en nuestro estudio, una sesión será un dia 1D).
2. La media y la StdDev del volumen profile de la sesión nos ayudarán a analizar correctamente las zonas.

Por tanto, una **zona de baja cotización** consiste en:
- Los niveles de precio que se encuentran N*StdDev(mean(volume_profile)) por debajo.

In [82]:
def data_from_mt5():
    # connect to MetaTrader 5
    if not mt5.initialize():
        print("initialize() failed")
        mt5.shutdown()

    # set time zone to UTC
    timezone = pytz.timezone("Etc/UTC")
    # create 'datetime' objects in UTC time zone to avoid the implementation of a local time zone offset
    utc_from = datetime(2010, 1, 1, tzinfo=timezone)
    utc_to = datetime(2024, 8, 1, tzinfo=timezone)

    # request AUDUSD ticks within 11.01.2020 - 11.01.2020
    ohlcv = mt5.copy_rates_range(TICKER, mt5.TIMEFRAME_M1, utc_from, utc_to)
    daily_ohlcv = mt5.copy_rates_range(TICKER, mt5.TIMEFRAME_D1, utc_from, utc_to)

    mt5.shutdown()

    df = pd.DataFrame(ohlcv)
    df['time']=pd.to_datetime(df['time'], unit='s')

    # adaptamos el dataframe D1 para luego hacer los analisis de las sesiones
    daily_df = pd.DataFrame(daily_ohlcv)
    daily_df['time']=pd.to_datetime(daily_df['time'], unit='s')

    df = df.set_index('time')
    del df['real_volume']
    del df['spread']

    daily_df = daily_df.set_index('time')
    del daily_df['real_volume']
    del daily_df['spread']

    return df, daily_df

def data_from_csv():
    df = pd.read_csv('C:/Users/iamfr/AlgoTrading/DATA/SI_Z_M1_200912010000_202406062259.csv', sep='\t')
    df['<DATE>'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'])
    del df['<TIME>']
    del df['<VOL>']
    del df['<SPREAD>']
    df.columns = ['time', 'open','high', 'low', 'close', 'tick_volume']
    df = df.set_index('time')

    daily_df = pd.read_csv('C:/Users/iamfr/AlgoTrading/DATA/SI_Z_Daily_200912010000_202406060000.csv', sep='\t')
    #df['<DATE>'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'])
    #del df['<TIME>']
    del daily_df['<VOL>']
    del daily_df['<SPREAD>']
    daily_df.columns = ['time', 'open','high', 'low', 'close', 'tick_volume']
    daily_df = daily_df.set_index('time')
    #daily_df = df.resample('1B').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'tick_volume': 'sum'})

    return df, daily_df

df, daily_df = data_from_csv()

df

,open,high,low,close,tick_volume
time,,,,,
2009-12-01 00:00:00,23.00,23.00,22.99,23.00,22
2009-12-01 00:01:00,23.01,23.02,23.00,23.00,67
2009-12-01 00:02:00,23.01,23.01,23.00,23.01,43
2009-12-01 00:03:00,23.01,23.02,23.01,23.01,13
2009-12-01 00:04:00,23.01,23.03,23.01,23.03,57
...,...,...,...,...,...
2024-06-06 22:55:00,31.30,31.30,31.29,31.29,42
2024-06-06 22:56:00,31.29,31.29,31.29,31.29,36
2024-06-06 22:57:00,31.29,31.29,31.29,31.29,21


In [55]:
def create_market_profile(data, getPOC=True):
    profile = data.groupby('close')['tick_volume'].sum().reset_index()
    total_volume = profile['tick_volume'].sum()
    profile['volume_cumsum'] = profile['tick_volume'].cumsum()

    value_area_cutoff = total_volume * 0.70
    value_area_df = profile[profile['volume_cumsum'] <= value_area_cutoff]
    POC = 0
    if getPOC:
        POC = profile.loc[profile['tick_volume'].idxmax(), 'close']
    else:
        POC = profile.loc[profile['tick_volume'].idxmin(), 'close']

    return profile, value_area_df, POC

def plot_market_profile(profile, value_area_df, POC):
    plt.figure(figsize=(10, 6))
    plt.barh(profile['close'], profile['tick_volume'], color='blue', edgecolor='black')
    plt.barh(value_area_df['close'], value_area_df['tick_volume'], color='green', edgecolor='black')
    plt.axhline(POC, color='red', linestyle='--', label=f'POC: {POC}')

    plt.xlabel('Volume')
    plt.ylabel('Price')
    plt.title(f'Market Profile')
    plt.legend()
    plt.show()

In [56]:
def get_no_fair_range_zone(df, threshold):
    # Crear una máscara booleana para identificar dónde 'tick_volume' es inferior al umbral
    df.sort_values(by=['close'])
    mask = df['tick_volume'] < threshold

    # Encontrar los índices donde empieza y termina cada zona
    rangos = []
    inicio = None
    rsize = 0
    rango_max = []

    for i in range(len(df)):
        if mask[i]:
            if inicio is None:  # Se inicia una nueva zona
                inicio = i
        else:
            if inicio is not None:  # Se cierra la zona actual
                rangos.append([inicio, i - 1])
                inicio = None

    # Si la última zona no se cierra explícitamente en el bucle
    if inicio is not None:
        rangos.append([inicio, len(df) - 1])
                 
    for rango in rangos:
        if rsize <= (rango[1] - rango[0]):  
              rsize = rango[1] - rango[0]
              rango_max = rango

    if len(rangos) < 1:
        return False
    return [df.loc[rango_max[0], 'close'], df.loc[rango_max[1], 'close']]

def get_max_vol_zone(df, threshold):
    # Crear una máscara booleana para identificar dónde 'tick_volume' es inferior al umbral
    df.sort_values(by=['close'])
    mask = df['tick_volume'] > threshold

    # Encontrar los índices donde empieza y termina cada zona
    rangos = []
    inicio = None
    rsize = 0
    rango_max = []

    for i in range(len(df)):
        if mask[i]:
            if inicio is None:  # Se inicia una nueva zona
                inicio = i
        else:
            if inicio is not None:  # Se cierra la zona actual
                rangos.append([inicio, i - 1])
                inicio = None

    # Si la última zona no se cierra explícitamente en el bucle
    if inicio is not None:
        rangos.append([inicio, len(df) - 1])
                 
    for rango in rangos:
        if rsize <= (rango[1] - rango[0]):  
              rsize = rango[1] - rango[0]
              rango_max = rango

    if len(rangos) < 1:
        return False
    return [df.loc[rango_max[0], 'close'], df.loc[rango_max[1], 'close']]

In [74]:
# iteramos cada dia, y cada zona
def get_no_fair_zone_by_day(df):
    out = []

    for index1, day in df.groupby(df.index.date):

        profile, value_area_df, POC = create_market_profile(day, False)
        mean = profile['tick_volume'].mean()
        stddev = profile['tick_volume'].std()
        threshold = mean + stddev * STD_MULTIPLIER

        no_fair_value_zone = get_no_fair_range_zone(profile, threshold)

        if no_fair_value_zone == False:
            output = {
                "time": index1,
                "min_zone_high": np.nan,
                "min_zone_low": np.nan,
                "MIN": np.nan,
            }
            out.append(output)
        else:
            output = {
                "time": index1,
                "min_zone_high": no_fair_value_zone[1],
                "min_zone_low": no_fair_value_zone[0],
                "MIN": POC,
            }
            out.append(output)

    return out

# iteramos cada dia, y cada zona
def get_max_vol_zone_by_day(df):
    out = []

    for index1, day in df.groupby(df.index.date):

        profile, value_area_df, POC = create_market_profile(day)
        mean = profile['tick_volume'].mean()
        stddev = profile['tick_volume'].std()
        threshold = mean + stddev * STD_MULTIPLIER

        max_value_zone = get_max_vol_zone(profile, threshold)

        if max_value_zone == False:
            output = {
                "time": index1,
                "max_zone_high": np.nan,
                "max_zone_low": np.nan,
                "POC": np.nan,
            }
            out.append(output)
        else:
            output = {
                "time": index1,
                "max_zone_high": max_value_zone[1],
                "max_zone_low": max_value_zone[0],
                "POC": POC,
            }
            out.append(output)

    return out

## Analisis de datos

A partir de aqui, ya disponemos de las herramientas de manipulación de los datos, como para poder empezar a analizarlos.

Disponemos de una lista "ZONES_BY_DAY", la qual nos permitirá referenciar las zonas de baja cotización por dia, y sus niveles de precio. Asi podemos hacer un analisis más comodo.

In [75]:
#zones = get_no_fair_zone_by_day(df)
max_zones = get_max_vol_zone_by_day(df)
df_max_zones = pd.DataFrame(max_zones)
#df_zones['time'].astype('datetime64[ns]')
df_max_zones = df_max_zones.set_index('time')
df_max_zones.index.astype('datetime64[ns]')

#zones = get_no_fair_zone_by_day(df)
min_zones = get_no_fair_zone_by_day(df)
df_min_zones = pd.DataFrame(min_zones)
#df_zones['time'].astype('datetime64[ns]')
df_min_zones = df_min_zones.set_index('time')
df_min_zones.index.astype('datetime64[ns]')

main_df_2 = pd.merge(df_max_zones, df_min_zones, on=daily_df.index)
main_df_2 = main_df_2.set_index('key_0')
main_df = pd.merge(daily_df, main_df_2, on=daily_df.index)
#daily_df.join(df_zones, on=daily_df.index , how='inner')
main_df = main_df.set_index('key_0')
main_df.dropna()
main_df

,open,high,low,close,tick_volume,max_zone_high,max_zone_low,POC,min_zone_high,min_zone_low,MIN
key_0,,,,,,,,,,,
2009.12.01,23.00,24.26,22.99,24.09,33033,23.64,23.61,23.63,23.53,23.31,23.42
2009.12.02,24.09,24.45,23.94,24.20,32991,24.29,24.29,24.29,24.43,24.30,23.95
2009.12.03,24.20,24.49,23.67,23.87,37566,24.18,24.12,24.14,24.47,24.30,24.46
2009.12.04,23.87,24.06,23.20,23.33,42983,23.29,23.24,23.40,24.04,23.61,23.78
2009.12.05,23.34,23.39,22.97,23.27,10580,23.31,23.29,23.29,23.20,22.98,23.03
...,...,...,...,...,...,...,...,...,...,...,...
2024.05.31,31.19,31.79,30.20,30.39,226013,30.44,30.39,31.22,31.05,30.49,30.81
2024.06.03,30.41,30.82,29.81,30.74,176930,30.54,30.41,30.51,30.40,29.85,29.95
2024.06.04,30.77,30.90,29.38,29.49,205818,29.56,29.51,29.53,30.89,29.98,30.85


### 1. Cierre de rango en la siguiente sesión

Analizaremos si en la siguiente sesión, el precio cierra el rango del dia anterior.

In [76]:
def num1_next_session_close_zone(df):
    counter = 0

    for i in range(len(df.index) - 2):
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        zone_high = df.iloc[i]['max_zone_high']
        zone_low = df.iloc[i]['max_zone_low']

        if zone_high <= high and zone_high >= low and zone_low >= low and zone_low <= high:
            counter = counter + 1

    return (counter / (len(df.index) - 1)) * 100

print("Porcentaje de cierre de la zona: ", num1_next_session_close_zone(main_df))

Porcentaje de cierre de la zona:  71.73962869585148


### 2. Cierre de medio rango en la siguiente sesion

Analizaremos si en la siguiente sesión, el precio cierra la midat del rango.

In [77]:
def num2_next_session_close_half_zone(df):
    counter = 0

    for i in range(len(df.index) - 2):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        zone_high = df.iloc[i]['max_zone_high']
        zone_low = df.iloc[i]['max_zone_low']
        zone_mid = ((zone_high - zone_low) / 2) + zone_low

        if apertura >= zone_high:
            # Mitad superior
            if zone_high <= high and zone_high >= low and zone_mid >= low and zone_mid <= high:
                counter = counter + 1

        if apertura <= zone_low:
            # Mitad inferior
            if zone_mid <= high and zone_mid >= low and zone_low >= low and zone_low <= high:
                counter = counter + 1

    return (counter / (len(df.index) - 1)) * 100

print("Porcentaje de cierre de media zona: ", num2_next_session_close_half_zone(main_df))

Porcentaje de cierre de media zona:  68.32454732981893


### 3. Desde apertura a extremo de zona

Analisis de la cantidad de veces que el precio recorre el rango entre la apertura de la sesion y un valor extremo.

Depende de si la apertura es por debajo o per encima de la zona de alta capitalización, buscaremos largos o cortos.

In [78]:
def num3_from_open_to_range_max_volume(df):
    counter = 0

    for i in range(len(df.index) - 2):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        zone_high = df.iloc[i]['max_zone_high']
        zone_low = df.iloc[i]['max_zone_low']

        if apertura >= zone_high:
            # Bajista
            if zone_low <= high and zone_low >= low:
                counter = counter + 1

        if apertura <= zone_low:
            # Alcista
            if zone_high <= high and zone_high >= low:
                counter = counter + 1

    return (counter / (len(df.index) - 1)) * 100

print("Porcentaje de Apertura-Extremo: ", num3_from_open_to_range_max_volume(main_df))

Porcentaje de Apertura-Extremo:  66.92642677057071


### 4. Recorrido a POC anterior session desde apertura 

Analisis de la cantidad de veces que se testea el valor POC de la anterior sesion.

In [79]:
def num4_POC_test(df):
    counter = 0

    for i in range(len(df.index) - 2):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        poc = df.iloc[i]['POC']

        if apertura >= poc:
            # Bajista
            if poc <= high and poc >= low:
                counter = counter + 1

        if apertura <= poc:
            # Alcista
            if poc <= high and poc >= low:
                counter = counter + 1

    return (counter / (len(df.index) - 1)) * 100

print("Porcentaje de test POC: ", num4_POC_test(main_df))

Porcentaje de test POC:  82.78707311482924


### 5. Buscamos zonas de baja probabilidad, como las de menor volumen en trasiego.

In [80]:
def low_prob_MIN(df):
    counter = 0

    for i in range(len(df.index) - 2):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        poc = df.iloc[i]['MIN']

        if high >= poc >= low:
            counter = counter + 1

    return (counter / (len(df.index) - 1)) * 100

def low_prob_YL(df):
    counter = 0

    for i in range(len(df.index) - 2):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        yl = df.iloc[i]['low']

        if high >= yl >= low:
            counter = counter + 1

    return (counter / (len(df.index) - 1)) * 100

def low_prob_YH(df):
    counter = 0

    for i in range(len(df.index) - 2):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        yh = df.iloc[i]['high']

        if high >= yh >= low:
            counter = counter + 1

    return (counter / (len(df.index) - 1)) * 100

def low_prob_YO(df):
    counter = 0

    for i in range(len(df.index) - 2):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        yo = df.iloc[i]['open']

        if high >= yo >= low:
            counter = counter + 1

    return (counter / (len(df.index) - 1)) * 100


print("Porcentaje de test MIN Volume: ", low_prob_MIN(main_df))
print("Porcentaje de test YL: ", low_prob_YL(main_df))
print("Porcentaje de test YH: ", low_prob_YH(main_df))
print("Porcentaje de test YO: ", low_prob_YO(main_df))

Porcentaje de test MIN Volume:  49.3696997478799
Porcentaje de test YL:  46.45885858354343
Porcentaje de test YH:  49.62181984872794
Porcentaje de test YO:  53.564061425624566
